In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.keras.layers as tfl

2022-12-06 14:18:46.862052: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.models import Model
import tensorflow.keras.optimizers as optimizers
import tensorflow.keras.metrics as metrics

In [3]:

x_train = np.load("./cleaned_data/small/x_train.npy")
y_train = np.load("./cleaned_data/small/y_train.npy")
train_mu = np.load("./cleaned_data/small/train_mu.npy")
train_std = np.load("./cleaned_data/small/train_std.npy")
x_val = np.load("./cleaned_data/small/x_val.npy")
y_val= np.load("./cleaned_data/small/y_val.npy")


In [4]:
# y = [no sack, sack, time to sack]
# x = (M, 23, 11)
x_train.shape

(5991, 23, 11)

In [5]:
import keras.backend as K

def my_loss(y_true, y_output):
    bce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    mse = tf.keras.losses.MeanSquaredError()
    
    # get mse of only true positives
    true_sack_mask = y_true[:,1]==1

    if len(y_true[true_sack_mask]) == 0:
        return bce(y_true[:,0:-1], y_output[:,0:-1])
    else:
        return bce(y_true[:,0:-1], y_output[:,0:-1]) + mse(y_true[true_sack_mask][:,-1], y_output[true_sack_mask][:,-1])

def bce_metric(y_true, y_output):
    return K.mean(K.binary_crossentropy(y_true[:,0:-1], y_output[:,0:-1], from_logits=False))

def mse_metric(y_true, y_output):
    # get mse of only true positives
    true_sack_mask = y_true[:,1]==1
    if len(y_true[true_sack_mask]) == 0:
        return 0.0
    else:
        return K.mean(K.square(y_true[true_sack_mask][:,-1] - y_output[true_sack_mask][:,-1]), axis=-1)
    # return K.mean(K.square(y_pred[:,-1] - y_true[:,-1]), axis=-1)
    
def accuracy_metric(y_true, y_output):
    preds = K.cast(K.argmax(y_output[:,0:-1], axis=-1), 'float32')
    return K.mean(K.cast(y_true[:,1] == preds, 'float32'))

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def recall(y_true, y_output):
    preds = K.cast(K.argmax(y_output[:,0:-1], axis=-1), 'float32')
    true_positives = K.sum(K.round(K.clip(y_true[:,1] * preds, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true[:,1], 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_output):
    preds = K.cast(K.argmax(y_output[:,0:-1], axis=-1), 'float32')
    true_positives = K.sum(K.round(K.clip(y_true[:,1] * preds, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(preds, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


In [6]:
''' 
Model

how cropping layer works - https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-cropping-layers-with-keras.md
'''

' \nModel\n\nhow cropping layer works - https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-cropping-layers-with-keras.md\n'

In [7]:
''' 
Output:
[prob of no sack, prob of sack, time till sack]

If predict no sack, doesn't matter what time is (recorded as -1 in training data)
'''

def createModel(input_shape = (23,7)):
    
    X = tfl.Input(input_shape)  # define the input to the model
    flat = tfl.Flatten(input_shape=(23, 7))(X)     # Flatten to pass into linear layers
    d1 = tfl.Dense(50, activation='relu')(flat)
    d3 = tfl.Dense(3,activation=None)(d1)
    
    # have layer (batch_size, 3). Want to take (b, [0,1]) and turn them into probabilities, and keep (b, [2]) as time
    # https://datascience.stackexchange.com/questions/86740/how-to-slice-an-input-in-keras
    intermediate = tfl.Reshape((3,1), input_shape=(3,))(d3)
    
    probs = tfl.Cropping1D(cropping=(0,1))(intermediate)
    probs = tfl.Reshape((2,), input_shape=(2,1))(probs)
    probs = tfl.Activation('softmax')(probs)
    
    time = tfl.Cropping1D(cropping=(2,0))(intermediate)
    time = tfl.Reshape((1,), input_shape=(1,1))(time)
    
    # concatenate the probabilities and predicted_time_to_sack back into one layer
    out = tfl.Concatenate(axis=-1)([probs, time])
    
    model = Model(inputs=X, outputs=out)        # create model
    
    return model
    

In [8]:
model = createModel()

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 23, 7)]      0           []                               
                                                                                                  
 flatten (Flatten)              (None, 161)          0           ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 50)           8100        ['flatten[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 3)            153         ['dense[0][0]']                  
                                                                                              

2022-12-06 14:18:53.491465: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
LEARNING_RATE = 0.001
BETA_1 = 0.9
BETA_2 = 0.999
EPS = 1e-07

opt = optimizers.Adam(
    learning_rate=LEARNING_RATE,
    beta_1=BETA_1,
    beta_2=BETA_2,
    epsilon=EPS)

model.compile(loss = my_loss, optimizer = opt, metrics = [accuracy_metric, bce_metric, mse_metric, recall, precision])

In [10]:
# Better optimizer

lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=LEARNING_RATE,
    decay_steps=10000,
    decay_rate=0.9)

scheduled_opt = optimizers.Adam(
    learning_rate=lr_schedule,
    beta_1=BETA_1,
    beta_2=BETA_2,
    epsilon=EPS)

In [27]:
NUM_EPOCHS = 30
history = model.fit(x_train[:,:,4:], y_train, epochs=NUM_EPOCHS)

Epoch 1/30
188/188 [==============================] - 1s 4ms/step - loss: 0.0442 - accuracy_metric: 1.0000 - bce_metric: 0.0070 - mse_metric: 0.0370 - recall: 0.8989 - precision: 0.8989
Epoch 2/30
188/188 [==============================] - 0s 2ms/step - loss: 0.0698 - accuracy_metric: 0.9995 - bce_metric: 0.0079 - mse_metric: 0.0621 - recall: 0.8706 - precision: 0.8777
Epoch 3/30
188/188 [==============================] - 0s 2ms/step - loss: 0.0796 - accuracy_metric: 0.9983 - bce_metric: 0.0111 - mse_metric: 0.0682 - recall: 0.8243 - precision: 0.8316
Epoch 4/30
188/188 [==============================] - 0s 2ms/step - loss: 0.0765 - accuracy_metric: 0.9987 - bce_metric: 0.0105 - mse_metric: 0.0657 - recall: 0.8426 - precision: 0.8484
Epoch 5/30
188/188 [==============================] - 0s 2ms/step - loss: 0.0658 - accuracy_metric: 0.9982 - bce_metric: 0.0093 - mse_metric: 0.0565 - recall: 0.7926 - precision: 0.8070
Epoch 6/30
188/188 [==============================] - 0s 2ms/step - lo

In [28]:
train_no_sack = np.round(np.sum(y_train[:,0])/len(y_train), 3)
print(f"Percentage of training plays with no sack = {train_no_sack}")

Percentage of training plays with no sack = 0.941


In [29]:
model.predict(x_train[0:5,:,4:])

1/1 [==============================] - 0s 34ms/step


array([[9.9999690e-01, 3.0919468e-06, 2.4979432e+00],
       [9.9997938e-01, 2.0623305e-05, 2.5297964e+00],
       [9.9890745e-01, 1.0925011e-03, 1.2628291e+00],
       [9.9559397e-01, 4.4059805e-03, 4.5964584e+00],
       [9.9681205e-01, 3.1879521e-03, 2.5961945e+00]], dtype=float32)

In [30]:
metrics_df = pd.DataFrame(history.history)


In [31]:
metrics_df.tail()

,loss,accuracy_metric,bce_metric,mse_metric,recall,precision
25,0.028437,1.000000,0.002019,0.026323,0.851064,0.851064
26,0.034247,1.000000,0.001990,0.032120,0.856383,0.856383
27,0.040685,1.000000,0.002005,0.038516,0.861702,0.861702
28,0.036739,0.999668,0.002972,0.033873,0.871011,0.872340
29,0.034295,0.999834,0.003132,0.031027,0.872340,0.871011


In [32]:
num_no_sack = np.round(np.sum(y_val[:,0])/len(y_val), 3)
print(f"Percentage of validation plays with no sack = {num_no_sack}")

Percentage of validation plays with no sack = 0.947


In [33]:
val_loss, cat_acc, val_bce, val_mse, val_recall, val_precision = model.evaluate(x_val[:,:,4:], y_val, verbose=2)

41/41 - 0s - loss: 2.8002 - accuracy_metric: 0.9146 - bce_metric: 0.7119 - mse_metric: 2.0123 - recall: 0.0618 - precision: 0.0569 - 101ms/epoch - 2ms/step


In [34]:

print(f"val loss = {val_loss}")
print(f"categorical accuracy = {cat_acc}")
print(f"val_bce = {val_bce}")
print(f"val_mse = {val_mse}")
print(f"val_recall = {val_recall}")
print(f"val_precision = {val_precision}")

val loss = 2.8001644611358643
categorical accuracy = 0.9146341681480408
val_bce = 0.7118798494338989
val_mse = 2.012299060821533
val_recall = 0.06178861856460571
val_precision = 0.056910574436187744


In [40]:
model.predict(x_val[:,:,4:])[:,0]

41/41 [==============================] - 0s 2ms/step


array([0.8872072 , 0.9867906 , 0.8555738 , ..., 0.99581367, 0.9926259 ,
       0.99644214], dtype=float32)

In [37]:
# https://www.kdnuggets.com/2021/02/saving-loading-models-tensorflow.html

model_string = f"models/third_model/weights_epochs{NUM_EPOCHS}"
model.save_weights(model_string)